# I- DESCRIPTION

In [ ]:
qMRinfo('inversion_recovery'); % Describe the model

# II- MODEL PARAMETERS
## a- Create object

In [ ]:
Model = inversion_recovery;

## Download sample data from OSF
> The current `Model` is an instance of `inversion_recovery` class.
 
You can manually download the sample data for `inversion_recovery` [by clicking here](https://osf.io/cmg9z/download?version=3).

In [ ]:
dataDir = downloadData(Model,pwd);

## b- Set protocol
> Protocol is set according to the example data
 
%          |- inversion_recovery object needs 2 protocol field(s) to be assigned:
%          |-   IRData
%          |-   TimingTable

In [ ]:
TI = [350.0000; 500.0000; 650.0000; 800.0000; 950.0000; 1100.0000; 1250.0000; 1400.0000; 1700.0000];
% TI(ms) is a vector of [9X1]
Model.Prot.IRData.Mat = [ TI];
%%   
TR = 2500;
Model.Prot.TimingTable.Mat = [ TR];
%%   

# III- FIT EXPERIMENTAL DATASET
## a- Load experimental data
%          |- inversion_recovery object needs 2 data input(s) to be assigned:
%          |-   IRData
%          |-   Mask

In [ ]:
 
% IRData.mat contains [128  128    1    9] data.
 load('inversion_recovery_data/IRData.mat');
% Mask.mat contains [128  128] data.
 load('inversion_recovery_data/Mask.mat');
 data.IRData= double(IRData);
 data.Mask= double(Mask);

## b- Fit experimental data
> This section will fit data.

In [ ]:
FitResults = FitData(data,Model,0);

## c- Show fitting results
> * Output map will be displayed.
 
> * If available, a graph will be displayed to show fitting in a voxel.

In [ ]:
qMRshowOutput(FitResults,data,Model);

## d- Save results
> * qMR maps are saved in NIFTI and in a structure `FitResults.mat` that can be loaded in qMRLab graphical user interface.
> * Model object stores all the options and protocol
> * These objects can be easily shared or be used for simulation.

In [ ]:
FitResultsSave_nii(FitResults);

# IV- SIMULATIONS
> This section can be executed to run simulations for vfa_t1.
 
## a- Single Voxel Curve
> Simulates single voxel curves:
 
       1. Use equation to generate synthetic MRI data
       2. Add rician noise
       3. Fit and plot curve

In [ ]:
      x = struct;
      x.T1 = 600;
      x.rb = -1000;
      x.ra = 500;
      % Set simulation options
      Opt.SNR = 50;
      Opt.T1 = 600;
      Opt.M0 = 1000;
      Opt.TR = 3000;
      Opt.FAinv = 180;
      Opt.FAexcite = 90;
      Opt.Updateinputvariables = false;
      % run simulation
      figure('Name','Single Voxel Curve Simulation');
      FitResult = Model.Sim_Single_Voxel_Curve(x,Opt);

## b- Sensitivity analysis
> Simulates sensitivity to fitted parameters: 
 
       1. Vary fitting parameters from lower (lb) to upper (ub) bound.
       2. Run Sim_Single_Voxel_Curve Nofruns times
       3. Compute mean and std across runs

In [ ]:
      %              T1            rb            ra            
      OptTable.st = [6e+02         -1e+03        5e+02]; % nominal values
      OptTable.fx = [0             1             1]; %vary T1...
      OptTable.lb = [0.0001        -1e+04        0.0001]; %...from 0.0001
      OptTable.ub = [5e+03         0             1e+04]; %...to 5000
       Opt.SNR = 50;
       Opt.Nofrun = 5;
      % run simulation
      SimResults = Model.Sim_Sensitivity_Analysis(OptTable,Opt);
      figure('Name','Sensitivity Analysis');
      SimVaryPlot(SimResults, 'T1' ,'T1' );